In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point
from shapely.wkt import loads
from scipy.io import loadmat
from scipy import spatial
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc
import os
import glob
from scipy.spatial.ckdtree import cKDTree
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils

In [3]:
def nearest_neighbours(points, coords, points_required = 1, max_distance = 250.):
    """

    :param points: array of points to find the nearest neighbour for
    :param coords: coordinates of points
    :param points_required: number of points to return
    :param max_distance: maximum search radius
    :return:
    """
    # Initialise tree instance
    kdtree = cKDTree(data=coords)

    # iterate throught the points and find the nearest neighbour
    distances, indices = kdtree.query(points, k=points_required,
                                      distance_upper_bound=max_distance)
    # Mask out infitnite distances in indices to avoid confusion
    mask = ~np.isfinite(distances)

    distances[mask] = np.nan

    return distances, indices

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=200.)

    near_ind = i[0]

    return grid_dict[var][near_ind]


In [4]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\HowardE_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


# bring in the rjmcmc data

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\combined\HE_rjmcmc_.nc"

# Read in the data and convert to geodataframe

rj_dat = netCDF4.Dataset(infile)

In [5]:
# Since we will be wanting to plot the sections lets first grid them

# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 10., 3.


In [6]:
# We will use the lines from the rj

lines = rj_dat['line'][:]

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\lci\hdf5"

gridded_vars ={}

#gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines,
#                                    resampling_method = 'linear', save_hdf5 = True,
#                                    return_dict = True, hdf5_dir = hdf5_dir)

for line in lines:
    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    f = h5py.File(infile, 'r')
    gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    f = None
    gc.collect()

c:\users\pcuser\onedrive\github\hydrogeol_utils\hydrogeol_utils\plotting_utils.py:757: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  datasets[item.name[1:]] = item.value
c:\users\pcuser\onedrive\github\hydrogeol_utils\hydrogeol_utils\plotting_utils.py:766: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  datasets['depth_of_investigation'] = item.value
c:\users\pcuser\onedrive\github\hydrogeol_utils\hydrogeol_utils\plotting_utils.py:760: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  datasets['easting'] = item.value
c:\users\pcuser\onedrive\github\hydrogeol_utils\hydrogeol_utils\plotting_utils.py:768: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  datasets['elevation'] = item.value
c:\users\pcuser\onedrive\github\hydrogeol_utils\hydrogeol_utils\plotting_utils.py:770: H5pyDeprecationWarning: dataset.value has been deprecated. Use datas

In [7]:
# Now we bring in the magnetics to plot

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\Magnetics\HE\HE_TMA_rtp_1VD.tif"

mag_dataset = rasterio.open(inRaster)

mag = mag_dataset.read(1)

mag[mag == mag_dataset.get_nodatavals()] = np.nan


In [8]:
# Now we bring in the magnetics to plot

inRaster = r"C:\Users\PCUser\Desktop\2017_HowardsEast_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)



In [16]:
# Bring in a suset of the points to sample

infile = r"C:\temp\output_picking.csv"

df = pd.read_csv(infile)

#df['top_conductor'] = np.nan

# Add the distance along the gridded lines to the dataframe

for index, row in df.iterrows():
    # get the line and coordinates
    line = row['line']
    
    xy = np.array([row[['easting','northing']].values])
        
    df.loc[index,'dist_along_line'] = xy_2_var(gridded_vars[line],
                                               xy,'grid_distances')


In [17]:
# This function stores the top of the conductor in the dataframe on a click
def onclick(event):
    if event.xdata != None and event.ydata != None:
        df.at[ind, 'top_conductor'] = df.loc[ind, 'elevation'] - event.ydata

In [153]:
# Create a generator to iterate through the pandas dataframe

def gen(df):
    for index, row in df.iterrows():
        yield index, row
        
cond_gen = gen(df[df['line'] == 115001])

# If you are starting at a known point

#for i in range(8):
#    next(cond_gen)

In [165]:
def extract_data():
    """
    FUnction for extracting all the AEM data from the netCDF files
    """

    freq = rj_dat['conducitivty_bin_count'][point_ind_rj].data.astype(np.float)

    cond_pdf = freq / freq.sum(axis =1)[0]

    cond_pdf[cond_pdf == 0] = np.nan
    
    cp_freq = rj_dat['change_point'][point_ind_rj].data.astype(np.float)
    
    cp_pdf = cp_freq / freq.sum(axis =1)[0]
    
    laybins = rj_dat['nlayer_bin_count'][point_ind_rj].data
    
    lay_prob = laybins / freq.sum(axis =1)[0]
    
    cond_cells = rj_dat["conductivity_cells"][:].data
    
    depth_cells = rj_dat['layer_top_depth'][point_ind_rj].data
    
    extent = [cond_cells.min(), cond_cells.max(), depth_cells.max(), depth_cells.min()]
    
    mean = rj_dat['conductivity_mean'][point_ind_rj].data
    p10 = rj_dat['conductivity_p10'][point_ind_rj].data
    p50 = rj_dat['conductivity_p50'][point_ind_rj].data
    p90 = rj_dat['conductivity_p90'][point_ind_rj].data
    
    lci_cond = lci_dat['conductivity'][point_ind_lci].data
    lci_depth_top = lci_dat['layer_top_depth'][point_ind_lci].data
    lci_doi = lci_dat['depth_of_investigation'][point_ind_lci].data
    
    misfit = rj_dat['misfit'][point_ind_rj].data
    sample_no = rj_dat['rj_sample_number'][:].data
    
    burnin = rj_dat["nburnin"][point_ind_rj].data
    nsamples = rj_dat['nsamples'][point_ind_rj].data
    nchains = rj_dat['nchains'][point_ind_rj].data
    
    line_ind = rj_dat['line_index'][point_ind_rj].data
    line = int(rj_dat['line'][line_ind].data)
    
    return {'conductivity_pdf': cond_pdf, "change_point_pdf": cp_pdf, "conductivity_extent": extent,
           'cond_p10': p10, 'cond_p50': p50, 'cond_p90': p90, 'cond_mean': mean, 'depth_cells': depth_cells,
           'nlayer_bins': laybins, 'nlayer_prob': lay_prob, 'nsamples': nsamples, 'ndata': rj_dat['n_data'][:].data,
           "nchains": nchains, 'burnin': burnin, 'misfit': misfit, 'sample_no': sample_no, 'cond_cells': cond_cells, 'lci_cond': lci_cond,
           'lci_depth_top': lci_depth_top, 'lci_doi': lci_doi, 'line': line}
    
def HE_plot(D, outfile = None):
    fig = plt.figure(figsize = (12,10))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '5in'

    ax1 = fig.add_axes([0.05, 0.35, 0.35, 0.6])
    ax2 = fig.add_axes([0.45, 0.35, 0.2, 0.6])
    ax3 = fig.add_axes([0.72, 0.6, 0.18, 0.18])
    ax4 = fig.add_axes([0.72, 0.35, 0.18, 0.18])
    ax5 = fig.add_axes([0.1, 0.2, 0.8, 0.05])
    ax6 = fig.add_axes([0.1, 0.1, 0.8, 0.1])
    ax7 = fig.add_axes([0.72, 0.82, 0.18, 0.18])
    cbar_ax1 = fig.add_axes([0.05, 0.28, 0.35, 0.01])
    cbar_ax2 = fig.add_axes([0.95, 0.1, 0.01, 0.2])
    
    panel_kwargs = [{'title': 'LCI residuals',
                      'color': 'black',
                      'ylabel': 'data residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 350.0,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': 2.,
                      'cmap': 'jet',
                      'ylabel': 'elevation_(mAHD)',
                      'vertical_exaggeration': 1.0}]


    # Plot probability map
    im = ax1.imshow(D['conductivity_pdf'], extent = D['conductivity_extent'],
                    aspect = 'auto', cmap = 'rainbow')
    #  PLot the median, and percentile plots
    ax1.plot(np.log10(D['cond_p10']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p10')
    ax1.plot(np.log10(D['cond_p90']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p90')
    ax1.plot(np.log10(D['cond_p50']), D['depth_cells'], c = 'k',label = 'p50')
    ax1.plot(np.log10(D['cond_mean']), D['depth_cells'], c = 'grey',label = 'mean')

    # for lci layered model we do some processing
    lci_expanded = np.zeros(shape=2 * len(D['lci_cond']) + 1,
                                 dtype=np.float)

    lci_expanded[1:] = np.repeat(D['lci_cond'], 2)

    depth_expanded = (np.max(D['lci_depth_top']) + 10) * np.ones(shape=len(lci_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(D['lci_depth_top'], 2)

    ax1.plot(np.log10(lci_expanded), depth_expanded, c = 'pink',
             linestyle = 'dashed', label = 'lci')
    ax1.plot(ax1.get_xlim(), [D['lci_doi'], D['lci_doi']], c = 'yellow',
             label = 'LCI doi')
    ax1.set_title('probability map')
    ax1.set_ylabel('depth (mBGL)')
    ax1.set_xlabel('log10 conductivity (S/m)')
    ax1.grid(which = 'both')
    ax1.set_xlim(D['conductivity_extent'][0], D['conductivity_extent'][1])
    ax1.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])
    ax1.legend()

    ax2.plot(D['change_point_pdf'], D['depth_cells'], label = 'P(change point)')
    ax2.set_ylim(ax2.get_ylim()[::-1])
    ax2.set_yticks(np.arange(0, 500, 20.))
    ax2.set_title('change point probability')
    ax2.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])
    ax2.legend()
    ax2.grid(which = 'both')

    ax3.imshow(mag, extent = [mag_dataset.bounds[0], mag_dataset.bounds[2],
                              mag_dataset.bounds[1], mag_dataset.bounds[3]],
              cmap= 'Greys', vmin = -2., vmax = 2.)
    ax3.set_xlim(D['easting'] - 3000., D['easting'] + 3000.)
    ax3.set_ylim(D['northing'] - 3000., D['northing'] + 3000.)
    ax3.plot(D['easting'],D['northing'], 'o', c = 'pink')
    # Ax3 will be our location
    sample = D['sample_no'][:]
    
    # Add the misfit
    for i in range(D['misfit'].shape[0]):
       
        misfits = D['misfit'][i]
        ax4.plot(sample, misfits/D['ndata'])

    ax4.plot([1, D['nsamples']], [1,1], 'k')
    ax4.plot([D['burnin'], D['burnin']],[0.1,1e4], 'k')
    ax4.set_xlim([1, D['nsamples']])
    ax4.set_ylim(0.1, 1e4)

    ax4.set_xscale('log')
    ax4.set_yscale('log')

    ax4.set_xlabel("sample #")
    ax4.set_ylabel("Normalised misfit")
    
    # conductivity plot
    
    ax7.imshow(np.log10(cond[10]), extent = [cond_dataset.bounds[0],
                                  cond_dataset.bounds[2],
                                  cond_dataset.bounds[1], 
                                  cond_dataset.bounds[3]],
              cmap = 'jet',
              vmin = np.log10(panel_kwargs[1]['vmin']*1000.),
              vmax = np.log10(panel_kwargs[1]['vmax']*1000.))
    
    ax7.set_xlim(D['easting'] - 3000., D['easting'] + 3000.)
    ax7.set_ylim(D['northing'] - 3000., D['northing'] + 3000.)
    ax7.plot(D['easting'],D['northing'], 'o', c = 'pink')

    fig.colorbar(im, cax=cbar_ax1, orientation='horizontal')
    

    res1 = plot_utils.plot_single_line(ax5, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax5.set_title('LCI data residuals')

    im2 = plot_utils.plot_grid(ax6, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])                                                     

    
    ax6.plot(df[df['line'] == line]['dist_along_line'].values,
             df[df['line'] == line]['top_conductor'].values,
               '+', c = 'pink')
    
    cb2 = fig.colorbar(im2, cax=cbar_ax2, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)
    
    ax5.set_xlim(dist - 1000., dist + 1000.)
    ax6.set_xlim(dist - 1000., dist + 1000.)

    # Add colorbars

    ax6.plot([dist, dist], [-500, 500], 'pink')
    
    return fig   

In [166]:
# Define some coordinates to investigate

ind, row = next(cond_gen)

print(ind)

point_ind_rj = np.where(rj_dat['fiducial'][:] == row['fiducial'])[0][0]

# Get the nearest lci point

easting, northing = row['easting'], row['northing']

# iterate throught the points and find the nearest neighbour
distance, point_ind_lci = kdtree.query([easting, northing], k=1,
                                  distance_upper_bound=200)

D = extract_data()

D['easting'], D['northing'] = easting, northing

line = D['line']

# Find distance along the lci section
dist = xy_2_var(gridded_vars[line],
                 np.array([[easting, northing]]),
                         'grid_distances')

fig = HE_plot(D)

cid = fig.canvas.mpl_connect('button_press_event', onclick)


#df.to_csv(r"C:\temp\output_picking.csv", index = False)

100


C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:153: RuntimeWarning: invalid value encountered in log10


In [139]:
ind

99

In [140]:
df.to_csv(r"C:\Temp\HE_sel.csv")